- 출처 : https://rfriend.tistory.com/279?category=675917


1. pivot(), pivot_table()
    - pivot(data, index = 컬럼명, columns = 컬럼명, values =컬럼명)
2. stack(), unstack() 
    - stack() : 모든 컬럼(명)을 melt (컬럼명 -> 값)
3. melt() : 특정 컬럼을 melt
    - melt(data, id_vars = [] )
4. wide_to_long() : 안 됨
5. crosstab() : 범주형 변수 빈도수
    - crosstab(df.col1, df.col2)

## 1. pivot(), pd.pivot_table()

In [2]:
import numpy as np
import pandas as pd

In [5]:
data = pd.DataFrame({'cust_id': ['c1', 'c1', 'c1', 'c2', 'c2', 'c2', 'c3', 'c3', 'c3'],
                     'prod_cd': ['p1', 'p2', 'p3', 'p1', 'p2', 'p3', 'p1', 'p2', 'p3'],
                     'grade' : ['A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B'],
                     'cost': [30, 10, 0, 40, 15, 30, 0, 0, 10]})

data

,cust_id,prod_cd,grade,cost
0,c1,p1,A,30
1,c1,p2,A,10
2,c1,p3,A,0
3,c2,p1,A,40
4,c2,p2,A,15
5,c2,p3,A,30
6,c3,p1,B,0
7,c3,p2,B,0
8,c3,p3,B,10


In [6]:
data.pivot(index = 'cust_id', 
           columns = 'prod_cd', 
           values = 'cost')

prod_cd,p1,p2,p3
cust_id,,,
c1,30,10,0
c2,40,15,30
c3,0,0,10


In [7]:
pd.pivot_table(data, index = 'cust_id', columns = 'prod_cd', values = 'cost')

prod_cd,p1,p2,p3
cust_id,,,
c1,30,10,0
c2,40,15,30
c3,0,0,10


In [11]:
# data.pivot()은 종종 화면에 에러나는 경우가 있다고 함. pd.pivot_table() 쓰자
data.pivot(index = ['cust_id','grade'], columns = 'prod_cd', values='cost')

ValueError: Length of passed values is 9, index implies 2

### index or columns 가 2개 이상인 경우

In [12]:
# 1) index가 2개 이상인 경우
pd.pivot_table(data, index = ['cust_id','grade'], columns = 'prod_cd', values='cost')

,prod_cd,p1,p2,p3
cust_id,grade,,,
c1,A,30,10,0
c2,A,40,15,30
c3,B,0,0,10


In [13]:
# 2) columns가 2개 이상인 경우
pd.pivot_table(data, index = 'cust_id', columns = ['grade','prod_cd'], values = 'cost')

grade       A                B           
prod_cd    p1    p2    p3   p1   p2    p3
cust_id                                  
c1       30.0  10.0   0.0  NaN  NaN   NaN
c2       40.0  15.0  30.0  NaN  NaN   NaN
c3        NaN   NaN   NaN  0.0  0.0  10.0

### aggfunc 옵션

In [15]:
# default : np.mean
pd.pivot_table(data, index = 'grade', columns = 'prod_cd', values = 'cost')

prod_cd,p1,p2,p3
grade,,,
A,35.0,12.5,15.0
B,0.0,0.0,10.0


In [16]:
pd.pivot_table(data, index = 'grade', columns = 'prod_cd', values = 'cost',
              aggfunc = np.sum)

prod_cd,p1,p2,p3
grade,,,
A,70,25,30
B,0,0,10


### margins = True 옵션

In [17]:
pd.pivot_table(data, index = 'grade', columns = 'prod_cd', values = 'cost',
              aggfunc = np.sum, margins = True)

prod_cd,p1,p2,p3,All
grade,,,,
A,70,25,30,125
B,0,0,10,10
All,70,25,40,135


---
## 2. stack(), unstack()
- stack() : 아래로 길게 쌓는다
- unstack() : 옆으로 길게 넓힌다

In [4]:
mul_index = pd.MultiIndex.from_tuples([('cust_1', '2015'), 
                                       ('cust_1', '2016'),
                                       ('cust_2', '2015'), 
                                       ('cust_2', '2016')])
mul_index

MultiIndex(levels=[['cust_1', 'cust_2'], ['2015', '2016']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [10]:
data = pd.DataFrame(data = np.arange(16).reshape(4,4),
                   index = mul_index,
                   columns = ['prod_1','prod_2','prod_3','prod_4'],
                   dtype = 'int')

data

prod_1  prod_2  prod_3  prod_4
cust_1 2015       0       1       2       3
       2016       4       5       6       7
cust_2 2015       8       9      10      11
       2016      12      13      14      15

### 1) stack()

In [12]:
# df -- stack() --> Series
data_stacked = data.stack()   # return Series
data_stacked

cust_1  2015  prod_1     0
              prod_2     1
              prod_3     2
              prod_4     3
        2016  prod_1     4
              prod_2     5
              prod_3     6
              prod_4     7
cust_2  2015  prod_1     8
              prod_2     9
              prod_3    10
              prod_4    11
        2016  prod_1    12
              prod_2    13
              prod_3    14
              prod_4    15
dtype: int32

In [13]:
data_stacked.index

MultiIndex(levels=[['cust_1', 'cust_2'], ['2015', '2016'], ['prod_1', 'prod_2', 'prod_3', 'prod_4']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1], [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]])

In [17]:
# indexing
data_stacked['cust_2']['2015'][['prod_1','prod_2']]

prod_1    8
prod_2    9
dtype: int32

### dropna 옵션

In [19]:
data.ix['cust_2','prod_4'] = np.nan
data

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


prod_1  prod_2  prod_3  prod_4
cust_1 2015       0       1       2     3.0
       2016       4       5       6     7.0
cust_2 2015       8       9      10     NaN
       2016      12      13      14     NaN

In [21]:
data.stack(dropna = True) # default

cust_1  2015  prod_1     0.0
              prod_2     1.0
              prod_3     2.0
              prod_4     3.0
        2016  prod_1     4.0
              prod_2     5.0
              prod_3     6.0
              prod_4     7.0
cust_2  2015  prod_1     8.0
              prod_2     9.0
              prod_3    10.0
        2016  prod_1    12.0
              prod_2    13.0
              prod_3    14.0
dtype: float64

In [43]:
data.stack(dropna = False) # default

cust_1  2015  prod_1     0.0
              prod_2     1.0
              prod_3     2.0
              prod_4     3.0
        2016  prod_1     4.0
              prod_2     5.0
              prod_3     6.0
              prod_4     7.0
cust_2  2015  prod_1     8.0
              prod_2     9.0
              prod_3    10.0
              prod_4     NaN
        2016  prod_1    12.0
              prod_2    13.0
              prod_3    14.0
              prod_4     NaN
dtype: float64

### 2) unstack()

In [24]:
data_stacked    # level = 3개인 MultiIndex
# 이 경우, level = -1, 0, 1 별로 어떤 level이 컬럼으로 이동하는지 보자

cust_1  2015  prod_1     0
              prod_2     1
              prod_3     2
              prod_4     3
        2016  prod_1     4
              prod_2     5
              prod_3     6
              prod_4     7
cust_2  2015  prod_1     8
              prod_2     9
              prod_3    10
              prod_4    11
        2016  prod_1    12
              prod_2    13
              prod_3    14
              prod_4    15
dtype: int32

In [32]:
data_stacked.unstack(level=0)

cust_1  cust_2
2015 prod_1       0       8
     prod_2       1       9
     prod_3       2      10
     prod_4       3      11
2016 prod_1       4      12
     prod_2       5      13
     prod_3       6      14
     prod_4       7      15

In [33]:
data_stacked.unstack(level=1)

2015  2016
cust_1 prod_1     0     4
       prod_2     1     5
       prod_3     2     6
       prod_4     3     7
cust_2 prod_1     8    12
       prod_2     9    13
       prod_3    10    14
       prod_4    11    15

In [35]:
# prod index가 컬럼으로
data_stacked.unstack(level=2)
#data_stacked.unstack(level=-1) # 같은 결과

prod_1  prod_2  prod_3  prod_4
cust_1 2015       0       1       2       3
       2016       4       5       6       7
cust_2 2015       8       9      10      11
       2016      12      13      14      15

In [37]:
data_stack_unstacked = data_stacked.unstack(level = 2)
data_stack_unstacked

prod_1  prod_2  prod_3  prod_4
cust_1 2015       0       1       2       3
       2016       4       5       6       7
cust_2 2015       8       9      10      11
       2016      12      13      14      15

In [40]:
# converting index to columns
data_stack_unstack_df = data_stack_unstacked.reset_index()
data_stack_unstack_df 

,level_0,level_1,prod_1,prod_2,prod_3,prod_4
0,cust_1,2015,0,1,2,3
1,cust_1,2016,4,5,6,7
2,cust_2,2015,8,9,10,11
3,cust_2,2016,12,13,14,15


In [42]:
# chaging columns' name
data_stack_unstack_df.rename(columns = {'level_0':'custID','level_1':'year'}, 
                             inplace=True)

data_stack_unstack_df

,custID,year,prod_1,prod_2,prod_3,prod_4
0,cust_1,2015,0,1,2,3
1,cust_1,2016,4,5,6,7
2,cust_2,2015,8,9,10,11
3,cust_2,2016,12,13,14,15


---
## 3. melt()

In [97]:
data = pd.DataFrame({'cust_ID' : ['C_001', 'C_001', 'C_002', 'C_002'],
                  'prd_CD' : ['P_001', 'P_002', 'P_001', 'P_002'],
                  'prc_cnt' : [1, 2, 3, 4],
                  'prc_cost' : [100, 200, 300, 400]})
data

,cust_ID,prd_CD,prc_cnt,prc_cost
0,C_001,P_001,1,100
1,C_001,P_002,2,200
2,C_002,P_001,3,300
3,C_002,P_002,4,400


In [61]:
pd.melt(data, id_vars = ['cust_ID', 'prd_CD'])

,cust_ID,prd_CD,variable,value
0,C_001,P_001,prc_cnt,1
1,C_001,P_002,prc_cnt,2
2,C_002,P_001,prc_cnt,3
3,C_002,P_002,prc_cnt,4
4,C_001,P_001,prc_cost,100
5,C_001,P_002,prc_cost,200
6,C_002,P_001,prc_cost,300
7,C_002,P_002,prc_cost,400


In [63]:
# variable, value 컬럼 이름 사용자 지정
pd.melt(data, 
        id_vars = ['cust_ID','prd_CD'],
        var_name = 'prc_CD',
        value_name = 'prc_value')

,cust_ID,prd_CD,prc_CD,prc_value
0,C_001,P_001,prc_cnt,1
1,C_001,P_002,prc_cnt,2
2,C_002,P_001,prc_cnt,3
3,C_002,P_002,prc_cnt,4
4,C_001,P_001,prc_cost,100
5,C_001,P_002,prc_cost,200
6,C_002,P_001,prc_cost,300
7,C_002,P_002,prc_cost,400


### pd.melt() vs pd.pivot_table
- melt()는 id가 컬럼으로 존재하지만, pivot_table()은 ID가 index로 들어감
(아래 예제 참고)

In [64]:
data

,cust_ID,prd_CD,prc_cnt,prc_cost
0,C_001,P_001,1,100
1,C_001,P_002,2,200
2,C_002,P_001,3,300
3,C_002,P_002,4,400


In [65]:
# melt()
data_melt = pd.melt(data, id_vars = ['cust_ID','prd_CD'],
                   var_name = 'prc_CD',
                   value_name = 'prc_value')
data_melt

,cust_ID,prd_CD,prc_CD,prc_value
0,C_001,P_001,prc_cnt,1
1,C_001,P_002,prc_cnt,2
2,C_002,P_001,prc_cnt,3
3,C_002,P_002,prc_cnt,4
4,C_001,P_001,prc_cost,100
5,C_001,P_002,prc_cost,200
6,C_002,P_001,prc_cost,300
7,C_002,P_002,prc_cost,400


In [66]:
data_melt.index

RangeIndex(start=0, stop=8, step=1)

In [68]:
data_melt.columns

Index(['cust_ID', 'prd_CD', 'prc_CD', 'prc_value'], dtype='object')

In [70]:
# pivot_table()
data_melt_pivot = pd.pivot_table(data_melt,
                                index = ['cust_ID','prd_CD'],
                                columns = 'prc_CD',
                                values = 'prc_value',
                                aggfunc = np.mean)
data_melt_pivot

prc_CD          prc_cnt  prc_cost
cust_ID prd_CD                   
C_001   P_001         1       100
        P_002         2       200
C_002   P_001         3       300
        P_002         4       400

In [71]:
data_melt_pivot.index

MultiIndex(levels=[['C_001', 'C_002'], ['P_001', 'P_002']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['cust_ID', 'prd_CD'])

In [72]:
data_melt_pivot.columns

Index(['prc_cnt', 'prc_cost'], dtype='object', name='prc_CD')

---
## 4. wide_to_long()

In [73]:
np.random.seed(10)

In [77]:
dict(zip(range(3), np.random.randn(3)))

{0: -0.008383849928522256, 1: 0.6213359738904805, 2: -0.7200855607188968}

In [79]:
data_wide = pd.DataFrame({"C1prd1" : {0 : "a", 1 : "b", 2 : "c"},
                          "C1prd2" : {0 : "d", 1 : "e", 2 : "f"},
                          "C2prd1" : {0 : 2.5, 1 : 1.2, 2 : .7},
                          "C2prd2" : {0 : 3.2, 1 : 1.3, 2 : .1},
                          "value" : dict(zip(range(3), np.random.randn(3)))
})
data_wide

,C1prd1,C1prd2,C2prd1,C2prd2,value
0,a,d,2.5,3.2,-0.174600
1,b,e,1.2,1.3,0.433026
2,c,f,0.7,0.1,1.203037


In [80]:
data_wide.index

Int64Index([0, 1, 2], dtype='int64')

In [84]:
data_wide["seq_no"] = data_wide.index
data_wide

,C1prd1,C1prd2,C2prd1,C2prd2,value,seq_no
0,a,d,2.5,3.2,-0.174600,0
1,b,e,1.2,1.3,0.433026,1
2,c,f,0.7,0.1,1.203037,2


In [87]:
data_long = pd.wide_to_long(data_wide, 
                            ["C1", "C2"], 
                            i="seq_no", 
                            j="prd")
data_long  # 안됨

,,C2prd1,C1prd2,value,C1prd1,C2prd2,C1,C2
seq_no,prd,,,,,,,


---
## 5. crosstab()
- '범주형 변수'를 교차분석 하여, 행, 열 요인 기준별로 빈도를 세어주는 도수분포표
- cf) 수치형 변수 -> pivot_table 이용

In [88]:
data = pd.DataFrame({'id': ['id1', 'id1', 'id1', 'id2', 'id2', 'id3'],
                  'fac_1': ['a', 'a', 'a', 'b', 'b', 'b'],
                  'fac_2': ['d', 'd', 'd', 'c', 'c', 'd']})
data

,id,fac_1,fac_2
0,id1,a,d
1,id1,a,d
2,id1,a,d
3,id2,b,c
4,id2,b,c
5,id3,b,d


In [89]:
pd.crosstab(data.fac_1, data.fac_2)

fac_2,c,d
fac_1,,
a,0,3
b,2,1


In [92]:
# Multi-index, Multi-level
pd.crosstab(data.id, [data.fac_1, data.fac_2])

fac_1  a  b   
fac_2  d  c  d
id            
id1    3  0  0
id2    0  2  0
id3    0  0  1

In [93]:
pd.crosstab([data.fac_1, data.fac_2], data.id)

id           id1  id2  id3
fac_1 fac_2               
a     d        3    0    0
b     c        0    2    0
      d        0    0    1

In [94]:
# 행, 열에 이름 부여
pd.crosstab(data.id, [data.fac_1, data.fac_2],
           rownames = ["id_num"],
           colnames =['a_b','c_d'])

a_b     a  b   
c_d     d  c  d
id_num         
id1     3  0  0
id2     0  2  0
id3     0  0  1

### margin 옵션

In [95]:
pd.crosstab(data.id, [data.fac_1, data.fac_2],
            margins = True)

fac_1  a  b    All
fac_2  d  c  d    
id                
id1    3  0  0   3
id2    0  2  0   2
id3    0  0  1   1
All    3  2  1   6

### normalized 옵션
- 구성 비율로 교챠표 만들기

In [96]:
pd.crosstab(data.id, [data.fac_1, data.fac_2],
            normalize = True)

fac_1    a         b          
fac_2    d         c         d
id                            
id1    0.5  0.000000  0.000000
id2    0.0  0.333333  0.000000
id3    0.0  0.000000  0.166667